In [ ]:
import torch
import matplotlib.pyplot as plt
import math
import numpy as np
from torch.func import grad
from tqdm import tqdm

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)

In [ ]:
n=10
step_size=0.1
gamma=1
u=1
torch.manual_seed(0)
g_init=torch.zeros(size=(n,n))
torch.nn.init.orthogonal_(g_init)
xi_init=torch.zeros(size=(n,n))
potential_func=lambda x:1-10*x[0,0]**2

In [ ]:
torch.manual_seed(0)

num_unif_sample=100000
unif_samples=[torch.nn.init.orthogonal_(torch.zeros(size=(n,n))) for _ in range(num_unif_sample)]
unif_samples=[g.numpy() for g in unif_samples]
weights=[np.exp(-potential_func(g)) for g in unif_samples]
weights=weights/np.mean(weights)

In [ ]:
unif_p_val=[g[0,0] for g in unif_samples]
plt.hist(unif_p_val, weights=weights, bins=50, range=[-1,1], density=True, label='hist of reject sample', alpha=0.8)


In [ ]:
g=torch.clone(g_init)
xi=torch.clone(xi_init)
grad_U=grad(potential_func)

g_list_one_chain=[]
for _ in tqdm(range(100000)):
    noise=torch.randn(n, n)
    noise=torch.tril(noise, -1)
    noise=noise-noise.permute(1,0)
    euclidean_grad=grad_U(g)
    manifold_grad=g.T@euclidean_grad-euclidean_grad.T@g
    xi=(1-gamma*step_size)*xi-step_size*manifold_grad+math.sqrt(2*gamma*step_size)*noise
    g=g@torch.linalg.inv(torch.eye(n, device=xi.device)-step_size*xi/2)@(torch.eye(n, device=xi.device)+step_size*xi/2)
    g_list_one_chain+=[g.clone().numpy()]

In [ ]:
cut_from=10000
g00_one_chain=[g[0,0] for g in g_list_one_chain][cut_from:]

plt.figure(figsize=(5,4))
plt.hist(g00_one_chain, density=True, bins=50, label='trajectory of one chain', alpha=0.5)
plt.hist(unif_p_val, weights=weights, bins=50, range=[-1,1], density=True, label='rejection sample', alpha=0.5)
plt.xticks(np.arange(-1, 1.1, 0.5), fontsize=18)
plt.yticks(np.arange(0, 1.6, 0.5), fontsize=18)
plt.xlabel('X[0,0]', fontsize=18)
plt.ylabel('Density', fontsize=18)
plt.legend(fontsize=14)
plt.savefig('hist_SO10.pdf', bbox_inches ='tight')

plt.figure(figsize=(5,4))
plt.plot(np.arange(6000)+len(g_list_one_chain)-6000, g00_one_chain[-6000:])
plt.xticks(np.arange(94000, 100001, 2000), fontsize=18)
plt.yticks(np.arange(-1, 1.1, 0.5), fontsize=18)

plt.ylabel('X[0,0]', fontsize=18)
plt.xlabel('#iter', fontsize=18)
plt.savefig('trajectory_SO10.pdf', bbox_inches ='tight')


In [ ]:
num_chain=10000
g_list=g_init.unsqueeze(0).repeat([num_chain, 1,1]).cuda()
xi_list=xi_init.unsqueeze(0).repeat([num_chain, 1,1]).cuda()

In [ ]:
torch.manual_seed(1)

g_memory=[]
xi_memory=[]
from torch.func import grad
grad_U=grad(potential_func)

for i in tqdm(range(2000)):
    # noise in lie algebra
    noise=torch.randn(num_chain,n, n).cuda()
    noise=torch.tril(noise, -1)
    noise=noise-noise.permute(0,2,1)

    euclidean_grad=torch.vmap(grad_U)(g_list)
    manifold_grad= torch.vmap(lambda g, G: g.T@G-G.T@g)(g_list, euclidean_grad)
    xi_list=(1-gamma*step_size)*xi_list-step_size*manifold_grad+math.sqrt(2*gamma*step_size)*noise
    g_list=torch.vmap(lambda g, xi: g@torch.linalg.inv(torch.eye(n, device=xi.device)-step_size*xi/2)@(torch.eye(n, device=xi.device)+step_size*xi/2))(g_list, xi_list)
    g_memory+=[g_list.cpu().numpy()]
    xi_memory+=[xi_list.cpu().numpy()]
    if i%100==0:
        plt.figure()
        plt.hist(g_list[:, 0, 0].cpu(), bins=50, density=True, range=[-1,1])
        plt.show()


In [ ]:

g00_indep_chains=np.array([g_list[:, 0, 0] for g_list in g_memory])
g00_indep_chains=g00_indep_chains[:100, :]

ncol = g00_indep_chains.shape[0]

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(projection="3d")

my_cmap = plt.cm.inferno

yticks = np.arange(ncol, step=1)

xbins = np.linspace(g00_indep_chains.min().min(), g00_indep_chains.max().max(), 100)
xcenter = np.convolve(xbins, np.ones(2), "valid")/2
xwidth = np.diff(xbins)

for i, ytick in enumerate(yticks):
    col =  g00_indep_chains[ytick, :]
    histvals, _ = np.histogram(col, bins=xbins, density=True)
    ax.bar(left=xcenter, height=histvals, width=xwidth, zs=ytick, zdir="y", color=my_cmap((len(yticks)-i)/len(yticks)), alpha=0.666, edgecolor="grey", linewidth=0.1)

ax.invert_yaxis()
plt.yticks([0, 25, 75, 100], fontsize=20)
plt.xticks([-1, -0.5, 0.5, 1], fontsize=20)
ax.set_zticks([0, 5, 10])
ax.set_zticklabels([0, 5, 10], fontsize=20)
ax.set_xlabel("X[0,0]", fontsize=25)
ax.set_ylabel("#iter", fontsize=25)
ax.set_zlabel("Density", fontsize=25)

fig.canvas.draw()
plt.tight_layout()

plt.savefig('hist3d_1st_half_SO10.pdf', bbox_inches ='tight')


In [ ]:

g00_indep_chains=np.array([g_list[:, 0, 0] for g_list in g_memory])
g00_indep_chains=g00_indep_chains[100:, :]

ncol = g00_indep_chains.shape[0]

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(projection="3d")

my_cmap = plt.cm.inferno

yticks = np.arange(ncol, step=10)

xbins = np.linspace(g00_indep_chains.min().min(), g00_indep_chains.max().max(), 100)
xcenter = np.convolve(xbins, np.ones(2), "valid")/2
xwidth = np.diff(xbins)

for i, ytick in enumerate(yticks):
    col =  g00_indep_chains[ytick, :]
    histvals, _ = np.histogram(col, bins=xbins, density=True)
    ax.bar(left=xcenter, height=histvals, width=xwidth, zs=ytick+100, zdir="y", color=my_cmap((len(yticks)-i)/len(yticks)), alpha=0.666, edgecolor="grey", linewidth=0.1)

ax.invert_yaxis()
plt.yticks([100, 500, 1500, 2000], fontsize=20)
plt.xticks([-1, -0.5, 0.5, 1], fontsize=20)
ax.set_zticks(np.arange(0, 1.6, 0.5))
ax.set_zticklabels(np.arange(0, 1.6, 0.5), fontsize=20)
ax.set_xlabel("X[0,0]", fontsize=25)
ax.set_ylabel("#iter", fontsize=25)
ax.set_zlabel("Density", fontsize=25)

fig.canvas.draw()
plt.tight_layout()


plt.savefig('hist3d_2nd_half_SO10.pdf', bbox_inches ='tight')


In [ ]:
plt.figure(figsize=(5,4))
plt.plot(np.abs(np.mean(g00_indep_chains, axis=1)))
plt.yscale('log')
plt.ylim([1e-3, 1])
plt.xlabel('#iter', fontsize=18)
plt.ylabel('mean of X[0,0] indep chains', fontsize=18)
plt.xticks([0, 500, 1000, 1500, 2000], fontsize=18)
plt.yticks(fontsize=18)
plt.savefig('mean_plot_SO10.pdf', bbox_inches ='tight')